**Created by:** Revekka Gershovich **When:** Dic 4, 2024 **Why:** To clean and aggregate election returns data for years 1824 to 1968 from ICPSR 1, United States Historical Election Returns

In [ ]:
import os
import os.path as path
import pandas as pd
import numpy as np
import re


In [ ]:
parent_dir = os.path.abspath("/Users/revekkagershovich/Dropbox (MIT)/StateLaws")
os.chdir(parent_dir)
assert os.path.exists(parent_dir), "parent_dir does not exist"
intermed_data_dir = "./2_data/2_intermediate/political_data"
assert os.path.exists(intermed_data_dir), "Data directory does not exist"
raw_data_dir = "./2_data/1_raw/political_data"
assert os.path.exists(raw_data_dir), "Data directory does not exist"

In [ ]:
df = pd.read_csv(path.join(raw_data_dir, "./ICPSR_election_returns/DS0001/00001-0001-Data.csv"))

In [ ]:
# df = pd.read_csv(path.join(raw_data_dir, "./ICPSR_election_returns/DS0171/00001-0171-Data.csv"))

In [ ]:
# df5 = pd.read_csv(path.join(raw_data_dir, "./ICPSR_election_returns/DS0005/00001-0005-Data.csv"))

In [ ]:
df['ICPR_STATE_CODE'].head()

In [ ]:
# Step 1: Get all column names from df
column_names = df.columns
print(len(column_names))

# Step 2: Filter columns that start with 'X' followed by three digits (X###)
pattern = re.compile(r"^X\d{3}_")
filtered_columns = [col for col in column_names if pattern.match(col)]
print(len(filtered_columns))
print(len(column_names) - len(filtered_columns))

# Step 3: Remove the 'X###_' prefix
cleaned_columns = {re.sub(r"^X\d{3}_", "", col) for col in filtered_columns}

# Step 4: Count unique column names after removing the prefix
num_unique_columns = len(cleaned_columns)

print(f"Total number of columns after 1st melt should be {num_unique_columns + (len(column_names) - len(filtered_columns))}")

In [ ]:
cleaned_columns

In [ ]:
df.head()

# Deciphering variable names

**1.** Since this dataset is provided in ASCII format with a SAS or SPSS setup files, I have extracted all the dataset into a csv format using a very niche R library called asciiSetupReader written specifically for extraction of pre-2000s dataset formatted in this weird way. As variable names in CSV, I used labels defined in the setup file. You can find this file in our StateLaws Dropbox: the path to the file is 1_code/similarity_code/Political_similarity_code/ICSPR_00001_to_csv.R

**2.** "Scope of Project" documentation for the study that can be found here: https://www.icpsr.umich.edu/web/ICPSR/studies/1. According to it "There is no actual codebook for this collection. Variable information is contained in the setup files." Thus, here I am making a codebook for naming conventions in my file so that if I or anyone else ever needs to go to the raw data, they would not have to spend hours figuring out what variable in the raw data mean. 

# Codebook for ICPSR 1, United States Historical Election Returns

## State and County Identifiers
| **Column Name**         | **Description**                                                                                     |
|-------------------------|-----------------------------------------------------------------------------------------------------|
| `ICPR_STATE_CODE`       | ICPSR standardized state code.                                                                      |
| `COUNTY_NAME`           | Standardized county name.                                                                           |
| `IDENTIFICATION_NUMBER` | Unique numeric identifier for each county, enabling consistent referencing.                         |

## Congressional District Numbers
| **Column Name**           | **Description**                                                                                   |
|---------------------------|---------------------------------------------------------------------------------------------------|
| `CONG_DIST_NUMBER_YYYY`   | Congressional district number for a specific year (e.g., `CONG_DIST_NUMBER_1825`). May indicate the number of districts for split counties. |

## Election Results

### General Format

X###_##_TYPE_RACE_PARTYCODE_VOTE

### Components
| **Component**     | **Description**                                                                                           |
|-------------------|---------------------------------------------------------------------------------------------------------|
| `X###`           | Election year (e.g., `X824` = 1824).                                                                      |
| `##`             | Election type/level: <br> **1** = Presidential, **2** = Gubernatorial, **3** = Congressional/House elections. |
| `TYPE`           | Type of election: <br> **G** = General, **M** = Midterm, **S** = Special.                                 |
| `RACE`           | Race type: <br> Examples: `PRES` = President, `GOV` = Governor.                                           |
| `PARTYCODE`      | Code representing the political party. See the attached party codes file for definitions (e.g., `0025` = National Republican). |
| `VOTE`           | Number of votes received by the candidate.                                                                |
| `TOTAL_VOTE`     | Total votes cast for the specific race or election.                                                       |

### Examples
| **Column Name**               | **Description**                                                                             |
|-------------------------------|---------------------------------------------------------------------------------------------|
| `X824_1_G_PRES_0025_VOTE`     | Votes for the National Republican candidate in the 1824 presidential general election.      |
| `X825_2_G_GOV_0659_VOTE`      | Votes for a specific party in the 1825 gubernatorial general election.                      |
| `X827_3_M_H_AL_9001_VOTE`     | Votes in a midterm House election in district `9001` for Alabama in 1827.                   |
| `X836_2_G_GOV_TOTAL_VOTE`     | Total gubernatorial votes in the 1836 general election.                                     |

## Handling Duplicate or Corrected Entries
| **Column Name Example**       | **Description**                                                                             |
|-------------------------------|---------------------------------------------------------------------------------------------|
| `X825_2_G_GOV_0659_VOTE.1`    | A vote for a second candidate from '0659' party in 1825 gubernatorial election.|
| `X831_3_M_H_AL_0100_VOTE.2`   | A duplicate or re-evaluated entry for midterm House votes in district `0100` for Alabama in 1831. |

## Important Notes
- **Corrections:** Some entries, such as Jackson County in Georgia (`ID: 1510`), should be corrected to `1570` when analyzing by county.
- **Missing Values:** For counties not reporting data or not participating in elections, identifiers like `98` (placeholders) are used.
- **Party Codes:** Refer to the party codes section of the documentation contained in /Users/revekkagershovich/Dropbox (MIT)/StateLaws/2_data/1_raw/political_data/ICPSR_election_returns/DS0204/00001-0204-Documentation.txt for the specific meaning of codes like `0025`, `0659`, etc. which represent political parties.

In [ ]:
df.head(20)

# Finding out what .1 .2 and .3 mean

# 1st Melt To Make Year a Separate Variable

In [ ]:
df.drop(columns=[col for col in df.columns if col.startswith('CONG')], inplace=True)

In [ ]:
# Step 1: Identify columns and group them by their suffix
# Add all variables starting with "CONG" to id_vars
id_vars = ['ICPR_STATE_CODE', 'COUNTY_NAME', 'IDENTIFICATION_NUMBER']
grouped_columns = {}

# Group columns by their suffix (everything after the first underscore and without the year part)
for col in df.columns:
    if col.startswith('X'):
        suffix = '_'.join(col.split('_')[1:])  # Extract the suffix after the first underscore
        if suffix not in grouped_columns:
            grouped_columns[suffix] = []
        grouped_columns[suffix].append(col)

In [ ]:
print(grouped_columns.keys())

In [ ]:
df.head()

In [ ]:
# Step 2: Reshape each group and combine into a single table
reshaped_dataframes = []

for suffix, cols in grouped_columns.items():
    # Reshape the group into long format
    temp_df = pd.melt(df, id_vars=id_vars, value_vars=cols,
                      var_name='year', value_name=suffix)
    # Extract the year and adjust to full year format
    temp_df['year'] = temp_df['year'].str.extract(r'X(\d+)').astype(int) + 1000
    reshaped_dataframes.append(temp_df)

In [ ]:
# Step 3: Merge all reshaped groups into a single DataFrame
final_df = reshaped_dataframes[0]
for additional_df in reshaped_dataframes[1:]:
    final_df = final_df.merge(additional_df, on=id_vars + ['year'], how='outer')

final_df = final_df[['year'] + [col for col in final_df.columns if col != 'year']]

In [ ]:
final_df.columns

In [ ]:
# temp = final_df[final_df['2_G_GOV_0026_VOTE.1'].notna()]

# print(temp['year'].unique())

In [ ]:
# temp = final_df[final_df['2_G_GOV_0659_VOTE.1'].notna()]

# print(temp['year'].unique())

In [ ]:
# Rename TOTAL_VOTE columns to match the party code format ('0000' instead of a party code)
renamed_columns = {col: col.replace("TOTAL_VOTE", "0000_VOTE") for col in final_df.columns if "TOTAL_VOTE" in col}

final_df = final_df.rename(columns=renamed_columns)

# Rename OTHER_VOTE columns to match the party code format ('0000' instead of a party code)
renamed_columns = {col: col.replace("OTHER_VOTE", "3000_VOTE") for col in final_df.columns if "OTHER_VOTE" in col}

final_df = final_df.rename(columns=renamed_columns)

In [ ]:
final_df.columns

# Multiple Candidates From A Single Party
Dealing with situation where multiple candidate exists in single elections for a single party, i.e. there are 3 Opposition Republicans running for 1825 CT elections. 

1. I compared the election of 1825 in CT to Wikipedia data and discovered that one of the parties had 3 candidates, and in the data this party had .1 and .2 prefixes. 



In [ ]:
# # Exploring why some variables have version with suffixes .1, .2, etc. in the dataset
# # Step 1: Filter the dataset for the year 1825
# elec_1825 = final_df[final_df['year'] == 1825]

# # Step 2: Select columns that include 'COUNTY_NAME' and those starting with '2_G_GOV'
# selected_columns = ['COUNTY_NAME'] + [col for col in final_df.columns if col.startswith("2_G_GOV")]

# # Step 3: Keep only the selected columns
# elec_1825 = elec_1825[selected_columns]

# # Step 4: Drop columns that contain only NA values
# elec_1825 = elec_1825.dropna(axis=1, how='all')

# # Display the resulting dataframe
# print(elec_1825)

# #0012  OLD REPUBLICAN 
# #0001 FEDERALIST
# #0659 Opposition Republicans

# total_votes = elec_1825["2_G_GOV_0000_VOTE"].sum()
# print(f"Total votes for 2_G_GOV_0000_VOTE: {total_votes}")

In [ ]:
# Step 1: Identify columns with suffixes .1, .2, .3, etc.
suffix_pattern = re.compile(r"(.*)\.(\d+)$")  # Matches columns ending in .1, .2, etc.
grouped_columns = {}

for col in final_df.columns:
    match = suffix_pattern.match(col)
    if match:
        base_name = match.group(1)  # Extract base column name (without suffix)
        if base_name not in grouped_columns:
            grouped_columns[base_name] = []
        grouped_columns[base_name].append(col)

# Step 2: Identify related columns and rename base columns with .0 postfix
for base_name in grouped_columns.keys():
    if base_name in final_df.columns:  # If the original base column exists
        final_df.rename(columns={base_name: base_name + ".0"}, inplace=True)
        grouped_columns[base_name].append(base_name + ".0")  # Include renamed base column

# Step 3: Create new summed columns
for base_name, related_columns in grouped_columns.items():
    final_df[base_name] = final_df[related_columns].sum(axis=1)

# Step 4: Drop all columns with suffixes .0, .1, .2, etc.
columns_to_drop = [col for col in final_df.columns if re.search(r"\.\d+$", col)]
final_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
list(final_df.columns)

# Melt 2
## Removing party from the name into a separate variable

In [ ]:
# Step 1: Identify columns and group them by their base name (e.g., `2_G_GOV_VOTE`)
id_vars = ['ICPR_STATE_CODE', 'COUNTY_NAME', 'IDENTIFICATION_NUMBER', 'year'] + [col for col in df.columns if col.startswith('CONG')]
grouped_columns = {}

# Group columns by removing the numeric segment before "_VOTE" (the second-to-last segment)
for col in final_df.columns:
    if '_VOTE' in col:  # Ensure we're only processing relevant columns
        parts = col.split('_')
        base_name = '_'.join(parts[:-2] + ['VOTE']) if parts[-2].isdigit() else col  # Remove numeric part before "VOTE"
        
        if base_name not in grouped_columns:
            grouped_columns[base_name] = []
        grouped_columns[base_name].append(col)

In [ ]:
print(grouped_columns.keys())

In [ ]:
# Step 2: Reshape each group and combine into a single table
reshaped_dataframes = []

for base_name, cols in grouped_columns.items():
    # Reshape the group into long format
    temp_df = pd.melt(final_df, id_vars=id_vars, value_vars=cols,
                      var_name='party', value_name=base_name)


    # Extract the 4-digit party code from column names
    extracted_party = temp_df['party'].str.extract(r'_(\d{4})_')
    temp_df['party'] = extracted_party[0]  # Get first column from extracted DataFrame

    reshaped_dataframes.append(temp_df)

In [ ]:
# Step 3: Merge all reshaped groups into a single DataFrame
final_df = reshaped_dataframes[0]
for additional_df in reshaped_dataframes[1:]:
    final_df = final_df.merge(additional_df, on=id_vars + ['party'], how='outer')

final_df = final_df[['party'] + [col for col in final_df.columns if col != 'party']]

Gusi

In [ ]:
final_df['party'].unique()

In [ ]:
final_df['year'].unique()

In [ ]:
final_df.pivot_table(index='COUNTY_NAME', columns='year', aggfunc='size', fill_value=0)

In [ ]:
final_df.columns

In [ ]:
final_df.rename(columns={'ICPR_STATE_CODE': 'ICPSR_state_code', 'COUNTY_NAME': 'county_name', 
                        'IDENTIFICATION_NUMBER': 'county_id'
}, inplace=True)

final_df['county_name'] = final_df['county_name'].str.title()

In [ ]:
# Create ICPSR to FIPS and ICPSR to State Name mappings
icpsr_to_fips = {
    1: 9,  2: 23, 3: 25, 4: 33, 5: 44, 6: 50, 11: 10, 12: 34, 13: 36, 14: 42, 21: 17,
    22: 18, 23: 26, 24: 39, 31: 19, 32: 20, 33: 27, 34: 29, 35: 31, 36: 38, 37: 46,
    40: 51, 41: 1, 42: 5, 43: 12, 44: 13, 45: 22, 46: 28, 47: 37, 48: 45, 49: 48,
    51: 21, 52: 24, 53: 40, 54: 47, 56: 54, 49: 48, 72: 41, 73: 53, 97: 97, 98: 11
}

# Add 'state_fips' column to cong_df based on 'ICPSR_state_code'
final_df['state_fips'] = final_df['ICPSR_state_code'].map(icpsr_to_fips)

In [ ]:
final_df.columns

In [ ]:
# Define the columns to keep (these should not be considered when checking for empty/zero rows)
columns_to_exclude = ['party', 'ICPSR_state_code', 'county_name', 'county_id', 'year', 'state_fips']

# Identify numeric columns that should be checked for being empty or zero
columns_to_check = [col for col in final_df.columns if col not in columns_to_exclude]

# Drop rows where all of the columns in `columns_to_check` are either NaN or 0
final_df = final_df[~(final_df[columns_to_check].isna() | (final_df[columns_to_check] == 0)).all(axis=1)]

In [ ]:
final_df.columns

In [ ]:
final_df.sample(20)

In [ ]:
# Define columns to group by (state-level aggregation by party-year pair)
groupby_columns = ['party', 'year', 'state_fips']

# Define columns to exclude from summation
columns_to_exclude = ['party', 'ICPSR_state_code', 'county_name', 'county_id', 'year', 'state_fips']

# Define columns to sum (all columns except those in columns_to_exclude)
columns_to_sum = [col for col in final_df.columns if col not in columns_to_exclude]


# Perform aggregation by summing vote counts at the state level
final_df = final_df.groupby(groupby_columns, as_index=False)[columns_to_sum].sum()

# Display the aggregated dataset
print(final_df.head())

## Next Steps

**1.** I should check that all the columns were melted correctly: it seems like some columns such as columns with different versions and such were dropped as well as AL columns. 

**2.** Start loading data in a loop for all states, and potentially concatinate it into one